In [1]:
#scrape urban turf
%pip install pandas
%pip install requests
import requests
%pip install beautifulsoup4
%pip install geopy
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import certifi

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.3 MB 7.4 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/11.3 MB 8.0 MB/s eta 0:00:02
   --------------- ------------------------ 4.5/11.3 MB 7.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.3 MB 7.5 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.3 MB 7.7 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.3 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.3 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 7.0 MB/s  0:00:01
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/12.9 MB 10.5 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/12.9 MB 8.4 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.9 MB 7.9 MB/s eta 0:00:01
   ---------

In [ ]:

page = requests.get(
    "https://dc.urbanturf.com/pipeline?search_string=&project_type=0&status=0&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes&limit=1000")
soup = BeautifulSoup(page.content, 'html.parser')
page_body = soup.body

labels = [item for item in page_body.findAll("p")]
list_lab = [i.text.strip() for i in labels][1:] 
list_lab = [word.replace('\n','') for word in list_lab]

torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using 'torep' value as keys
pattern = re.compile("|".join(torep.keys())) #joins with "|"

desc = []

for i in range(0, len(list_lab)):
    a = list_lab[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #substitute the pattern in each value of the list with ""
    desc.append(text)

seq = range(0, len(desc))

urban_turf= pd.DataFrame(
    {"Address": [desc[i] for i in seq[0::5]],
     "Neighborhood": [desc[i] for i in seq[1::5]],
     "Project.Type":[desc[i] for i in seq[2::5]],
     "Status":[desc[i] for i in seq[3::5]],
     "Size":[desc[i] for i in seq[4::5]],
     "Pipeline Link": None,
     'Neighborhood:':None,
     'Pipeline':None,
     'Project type:': None, 
     'No. of units:': None, 
     'Types of units:': None, 
     'Unit sizes:':None, 
     'Amenities:':None, 
     'Pricing:':None, 
     'Website:':None, 
     'Architect:':None, 
     'Last updated:':None, 
     'Address:':None, 
     'Status:':None, 
     'First move-ins:':None, 
     "Latitude":None, 
     "Longitude":None})    

#can I get the pipeline link from scraping above?

def search_pipe_link(row):
    ut_address_col = str(row['Address'])
    projecttype_col = str(row['Project.Type'])
    status_col = str(row['Status'])
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + ut_address_col.replace(" ", "_" ) + "&project_type=" + projecttype_col.replace(" ", "+").replace("&", "%26") + "&status=" + status_col.replace(",", "%2C").replace(" ", "+") + "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    time.sleep(1.25)
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a'py ).get('href')
        return tag
    else:
        return None

urban_turf['Pipeline Link'] = urban_turf.apply(search_pipe_link, axis=1)

'''
#do i still need this?
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

#if there is a pipeline link

def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in list(len(txt))] #need to expand range?
    txt = [txt[i][1] for i in list(len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)


'''
'''
for i in range(0, len(urban_turf.Address)):
    if type(urban_turf["Pipeline Link"][i]) == str:
    
        page = requests.get(urban_turf["Pipeline Link"][i])
        soup = BeautifulSoup(page.content, 'html.parser')
        page_body = soup.body
        table = soup.find_all('div', class_ = 'block') 
        txt= str(table)
        #every <span class> 
        #keep everything in between <span class="label">Project type:</span> and </p>

        txt = txt.replace('\r', '').replace('\n', '')

        #remove any instances where there are two spaces +
        for k in range(0,100):
            txt = txt.replace("  ", "")
        #remove all new lines
        txt = txt.split("<p>")  

        txt = [x for x in txt if '<span class="label">' in x] #this is the problem
        colnames = []
        values = []

        for j in range(0, len(txt)):
            a = txt[j].split("</span>")

            a[0] = re.sub('<span class="label">', "", a[0])

            colnames.append(a[0])

            a[1] = re.sub("</p>.*", "", a[1])
            a[1] = re.sub("\t{4,5}", "", a[1])
            a[1] = re.sub("<br/>", ",", a[1])
            a[1] = re.sub("\,.*DC", ", Washington, DC,", a[1])
            a[1] = re.sub('<a href="', "", a[1])
            a[1] = re.sub('" style.*', "", a[1])
            a[1] = re.sub('First', "1st", a[1])

            values.append(a[1])

        df_dictionary = pd.DataFrame([ dict(zip(colnames, values))])  #want to add this to urban_turf

        #urban_turf = pd.concat([urban_turf, df_dictionary],  ignore_index=True) #this adds it below instead of merging it 
        
        for key, value in df_dictionary.items():
            if key in column_names:
                urban_turf[key][i] = df_dictionary[key][0]
                  





#scrape pipeline links
for i in range(0, len(urban_turf.Address)):
    if type(urban_turf["Pipeline Link"][i]) == str:
    
        page = requests.get(urban_turf["Pipeline Link"][i])
        soup = BeautifulSoup(page.content, 'html.parser')
        page_body = soup.body
        table = soup.find_all('div', class_ = 'block') 
        txt= str(table)
        #every <span class> 
        #keep everything in between <span class="label">Project type:</span> and </p>

        txt = txt.replace('\r', '').replace('\n', '')

        #remove any instances where there are two spaces +
        for k in range(0,100):
            txt = txt.replace("  ", "")
        #remove all new lines
        txt = txt.split("<p>")  

        txt = [x for x in txt if '<span class="label">' in x] #this is the problem
        colnames = []
        values = []

        for j in range(0, len(txt)):
            a = txt[j].split("</span>")

            a[0] = re.sub('<span class="label">', "", a[0])

            colnames.append(a[0])

            a[1] = re.sub("</p>.*", "", a[1])
            a[1] = re.sub("\t{4,5}", "", a[1])
            a[1] = re.sub("<br/>", ",", a[1])
            a[1] = re.sub("\,.*DC", ", Washington, DC,", a[1])
            a[1] = re.sub('<a href="', "", a[1])
            a[1] = re.sub('" style.*', "", a[1])
            a[1] = re.sub('First', "1st", a[1])

            values.append(a[1])

        df_dictionary = pd.DataFrame([ dict(zip(colnames, values))])  #want to add this to urban_turf

        #urban_turf = pd.concat([urban_turf, df_dictionary],  ignore_index=True) #this adds it below instead of merging it 
        
        for key, value in df_dictionary.items():
            if key in column_names:
                urban_turf[key][i] = df_dictionary[key][0]
                  
    else:
        continue

def get_lat_long(address):
    geolocator = Nominatim(user_agent="my_geocoder_app")
    time.sleep(1.1)
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return None
    
urban_turf['Latitude'] = ""
urban_turf['Longitude'] = ""

for i in range(0, len(urban_turf)):
    try:
        c = urban_turf['Address:'][i]
        d = re.sub("-[\d]*", "", c)
        a = get_lat_long(d)
        urban_turf['Latitude'][i] =  a[0]
        urban_turf['Longaitude'][i] =  a[1]

    except Exception:
        pass
urban_turf['No. of units:'] = pd.to_numeric(urban_turf['No. of units:'], errors='coerce')
'''

'\nfor i in range(0, len(urban_turf.Address)):\n    if type(urban_turf["Pipeline Link"][i]) == str:\n    \n        page = requests.get(urban_turf["Pipeline Link"][i])\n        soup = BeautifulSoup(page.content, \'html.parser\')\n        page_body = soup.body\n        table = soup.find_all(\'div\', class_ = \'block\') \n        txt= str(table)\n        #every <span class> \n        #keep everything in between <span class="label">Project type:</span> and </p>\n\n        txt = txt.replace(\'\r\', \'\').replace(\'\n\', \'\')\n\n        #remove any instances where there are two spaces +\n        for k in range(0,100):\n            txt = txt.replace("  ", "")\n        #remove all new lines\n        txt = txt.split("<p>")  \n\n        txt = [x for x in txt if \'<span class="label">\' in x] #this is the problem\n        colnames = []\n        values = []\n\n        for j in range(0, len(txt)):\n            a = txt[j].split("</span>")\n\n            a[0] = re.sub(\'<span class="label">\', "", a

In [781]:
def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]

    key = [txt[i][0] for i in list(range(0,len(txt)))] #need to expand range?
    txt = [txt[i][1] for i in list(range(0,len(txt)))]
    #key = [x.replace('<span class="label">', "") for x in key]
    #df_dictionary = pd.DataFrame([ dict(zip(key, txt))])

    
    #for key, txt in df_dictionary.items():
    #        if key in column_names:
    #           urban_turf[key][row] = df_dictionary[key][0]

#urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)

In [788]:
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

#if there is a pipeline link

def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in list(len(txt))] #need to expand range?
    txt = [txt[i][1] for i in list(len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)

KeyError: "None of [Index([                                       'Valen',\n                                   'National Landing',\n                                             'Rental',\n                              'Delivered, Leasing Up',\n                                         '338  units',\n       'https://dc.urbanturf.com/pipeline/1255/Valen',\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                            

In [ ]:
def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in range(0, len(txt))] 
    txt = [txt[i][1] for i in range(0, len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

In [708]:
a[0]

'Project type:'

In [694]:
urban_turf['No. of units:'] = a[1]

In [ ]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,Valen,National Landing,Rental,"Delivered, Leasing Up",338 units,https://dc.urbanturf.com/pipeline/1255/Valen,None,None,Rental apart...,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
1,Bryant Street Development,Edgewood,Rental,"Under Construction, Leasing Up",1650 units,https://dc.urbanturf.com/pipeline/457/Bryant_S...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
2,Fox 5 Headquarters Redevelopment,Friendship Heights,Rental,"Under Construction, Not Yet Leasing",210 units,https://dc.urbanturf.com/pipeline/834/Fox_5_He...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
3,Barry Farm Redevelopment,Barry Farm,Condos & Rental apartments,"Under Construction, Leasing Up","1,000",https://dc.urbanturf.com/pipeline/579/Barry_Fa...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
4,Kiley Apartments,Southwest,Rental,"Delivered, Leasing Up",330 units,https://dc.urbanturf.com/pipeline/446/Kiley_Ap...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
673,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,None,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
674,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
675,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,None,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None


urban_turf

In [ ]:
def search_pipe_line(search, projecttype, status):
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + search + "&project_type=" + projecttype + "&status=" + status + "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a').get('href')
        return tag
    else:
        return None

In [ ]:
torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using value as keys
pattern = re.compile("|".join(torep.keys())) #joins

for i in range(0, len(list)):
    a = list[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #re.escape - remove extra values, #substitute the pattern in each value of the list (a)
    desc.append(text)